# Using *Key Point Summarization* service for analyzing and finding insights in a survey data 
When you have a large collection of texts representing people’s opinions (such as product reviews, survey answers or social media), it is difficult to understand the key issues that come up in the data. Going over thousands of comments is prohibitively expensive.  Existing automated approaches are often limited to identifying recurring phrases or concepts and the overall sentiment toward them, but do not provide detailed or actionable insights. 

In this tutorial you will gain hands-on experience in using *Key Point Summarization* (KPS) for analyzing and deriving insights from open-ended comments.  

The data we will use is [a community survey conducted in the city of Austin](https://data.austintexas.gov/dataset/Community-Survey/s2py-ceb7). In this survey, the citizens of Austin were asked "If there was ONE thing you could share with the Mayor regarding the City of Austin (any comment, suggestion, etc.), what would it be?". 

## 1. Initialization

### 1.1 Setup

Lets first import all required packages for this tutorial and initialize the *Key Point Summarization* client. The client prints information using the logger and a suitable verbosity level should be set. The client object is configured with an API key. It should be  retrieved from the [Project Debater Early Access Program](https://early-access-program.debater.res.ibm.com/) site.  In the code bellow it is passed by the enviroment variable *KPS_API_KEY* (you may also modify the code and place the api-key directly).

In [35]:
from debater_python_api.api.clients.keypoints_client import KpsClient, KpsJobFuture
from debater_python_api.api.clients.key_point_summarization.KpsResult import KpsResult
import os
import pandas as pd
import json 

pd.set_option('display.max_rows', None)
KpsClient.init_logger()
api_key = os.environ['KPS_API_KEY']
host = 'https://keypoint-matching-backend.debater.res.ibm.com'
keypoints_client = KpsClient(api_key, host)

### 1.2 Read the data
Let's read the data from *dataset_austin.csv* file, which holds the Austin survey dataset, and print the first comment.

In [3]:
comments_df = pd.read_csv('./dataset_austin.csv')

print(f'There are {len(comments_df)} comments in the dataset')
print(dict(comments_df.iloc[0,:]))

There are 3187 comments in the dataset
{'id': 1, 'year': 2016, 'text': "Dissatisfied traffic and with traffic, timing of street lights.  EXTREMELY dissatisfied with cit govt. interfering in local businesses (Uber/Lyft, income property owners).  Also, extremely dissatisfied with all the free handouts to people who are perfectly capable of earning their own money.  I'm very dissatisfied with the liberal leaning local politicians."}


Each comment has a unique_id 'id', a 'text' and a 'year'. We will first remove all comments with text longer than 3000 characters since this is a system's limit, and all empty comments. Then we will filter the comments and take the ones from 2016. 

The *Key Point Summarization* service is able to run over hundreds of thousands of comments. However, to provide good user experience the KPS evaluation service is limited to 1000 comments per run, each comment with up to 3000 chars. You may request to increase this limit if needed. 

We will choose a sample of 1000 comments  from 2016 for our analysis.

In [4]:
comments_df = comments_df.dropna()
comments_df = comments_df[comments_df.text.apply(lambda x: 0<len(str(x))<=3000)]

comments_2016_df = comments_df[comments_df['year'] == 2016]
sample_size = 1000
comments_2016_sample_df = comments_2016_df.sample(n = sample_size, random_state = 1)

## 2. Run the full KPS flow and generate results
The simplest way to run KPS is using the method run_full_kps_flow, which serves as an excellent starting point. To do so, you only need to provide a collection of textual comments and a distinct domain name (comprised of alphanumeric characters, spaces, or underscores). If you wish to reuse a domain from a previous run, you will first need to delete it via **keypoints_client.delete_domain_cannot_be_undone(domain=domain)**. 

Using this method, KPS extracts the key points from the provided data and matches each sentence in the input comments with its corresponding matching key points.

By default, the service performs stance analysis: it runs for positive (pro) and for negative (con) sentences seperately, and returns a merged result containing key points from both stances. To disable the stance analysis and run on all sentences together, add the parameter **stance=no-stance** to the **run_full_kps_flow** method.   

In [7]:
domain = "austin_demo_full"
comments_texts_2016 = list(comments_2016_sample_df['text'])
kps_result_2016 = keypoints_client.run_full_kps_flow(domain, comments_texts_2016)

2023-06-04 22:35:31,349 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/domains
2023-06-04 22:35:32,194 [INFO] keypoints_client.py 113: created domain: austin_demo_full with domain_params: None
2023-06-04 22:35:32,198 [INFO] keypoints_client.py 158: uploading 1000 comments in batches
2023-06-04 22:35:32,202 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 22:35:33,349 [INFO] keypoints_client.py 174: uploaded 1000 comments, out of 1000
2023-06-04 22:35:33,351 [INFO] keypoints_client.py 137: waiting for the comments to be processed
2023-06-04 22:35:33,352 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 22:35:33,984 [INFO] keypoints_client.py 187: domain: austin_demo_full, comments status: {'processed_comments': 0, 'processed_sentences': 0, 'pending_comment

Stage 1/2: |███████████████████████████████████---------------| 70.6% Complete



2023-06-04 22:39:02,836 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 22:39:03,445 [INFO] keypoints_client.py 635: job_id 647ce79ef0128c554c89b7f0 is running, progress: {'total_stages': 2, 'stage_1': {'inferred_batches': 17, 'total_batches': 17, 'batch_size': 2000}, 'stage_2': {'inferred_batches': 0, 'total_batches': 1, 'batch_size': 2000}}


Stage 2/2: |--------------------------------------------------| 0.0% Complete



2023-06-04 22:39:33,450 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 22:39:35,365 [INFO] keypoints_client.py 638: job_id 647ce79ef0128c554c89b7f0 is done, returning result
2023-06-04 22:39:35,598 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-04 22:39:36,878 [INFO] keypoints_client.py 408: domain: austin_demo_full was deleted


kps_result_2016 is a KpsResult object. Let's print top 20 key points, and the top three matched sentences per key point:

In [9]:
kps_result_2016.print_result(n_sentences_per_kp = 3, title = "Austin sample 2016 full flow", n_top_kps = 20)

Austin sample 2016 full flow results, stance: pro and con 
Coverage (all comments): 66.80
Displaying 20 key points out of 60:
185 - Improve traffic flow. - con
	- The traffic on Cameron Road was worsened by the addition of bike lanes.
	- Fix the traffic - you keep encouraging growth of the city without the infrastructure to
	  support that growth.
	- Get the bikes off the streets and improve traffic flow in and out of the city.
167 - We really need to improve our public transportation. - con
	- Need to rework zoning to improve traffic and add more public transit, especially to
	  suburbs.
	- The city needs a more comprehensive and better connected transportation network
	  including bus, rail, bike, pedestrian access.
	- Austin needs to get serious about alternatives to driving including real mass transit,
	  bike, and pedestrian facilities.
150 - Improve affordable housing/living. - con
	- Affordable housing is also a problem- the rents are getting too high and are hard for
	  familie

In the second line we can see that 67% of the comments were matched, i.e., had at least one sentence matched to at least one key point.
For each key point, this method prints the number of matched sentences within the input comments (including the key point itself), its stance (pro or con) and the top matching sentences.

## 3. KpsResult Object and result processing


The *KpsResult* object stores all the information about the job (or merged jobs). It can be used to generate several types of reports and to compare different results. All the reports generated in this section are available in the "kps_results" folder.

The KpaResult can be saved to file and loaded from file via the load and save methods:

In [11]:
json_file = "kps_results/merged_austin_results.json"
kps_result_2016.save(json_file) 
kps_result_2016 = KpsResult.load(json_file)

2023-06-04 22:54:21,714 [INFO] KpsResult.py 93: Writing results to: kps_results/merged_austin_results.json
2023-06-04 22:54:21,755 [INFO] KpsResult.py 104: Loading results from: kps_results/merged_austin_results.json


The KpsResult stores all the generated key points and all the sentences that are matched to them. Based on that, several reports and analyses can be generated.

### 3.1 Result Summary
The result summary presentend in the attribute *summary_df* displays the aggregated information per key point:

In [12]:
kps_result_2016.summary_df

,key_point,#comments,comments_coverage,#sentences,sentences_coverage,stance,kp_id,parent_id,n_comments_subtree
0,Improve traffic flow.,169,0.169,185,0.102041,con,0,None,220
1,We really need to improve our public transport...,153,0.153,167,0.092113,con,1,None,153
2,Improve affordable housing/living.,127,0.127,150,0.082736,con,2,None,154
3,PROPERTY TAXES ARE TOO HIGH.,83,0.083,96,0.052951,con,3,None,110
4,Please plan better for growth on our roadways!,78,0.078,85,0.046884,con,4,0,78
5,Buying a home in S Austin is prohibitive.,56,0.056,62,0.034197,con,5,2,56
6,driving in Austin is terrible,47,0.047,51,0.028130,con,6,0,47
7,Water costs too much.,44,0.044,50,0.027579,con,7,3,52
8,Don't let Austin become Houston with overdevel...,35,0.035,36,0.019857,con,10,None,35
9,Better pedestrian and biking lifestyle options.,34,0.034,36,0.019857,con,11,None,34


The report displays:  
- `key_point`: the list of generated key points, sorted by their saliance..
- `#comments`: the number of comments matched to the kry point (comments that have at least one sentence matched to the key point).  
- `comments_coverage`: the percentage of comments matched to it (out of the entire set of comments sent to the job).  
- `#sentences`: the number of the sentences matched to the key point.  
- `sentences_coverage`: the percentage of the sentences matched to the key point. 
- `stance`: the key point stance (if stance analysis was performed).
- `kp_id`: the key point's id.  
- `parent_id`: A The analysis also creates a key point tree-structred hierarchy. The parent_id column shows the kp_id of the parent of the key point. For example, the key point *BIKE LANES HURT TRAFFIC.* is under the parent key point *TRAFFIC NEEDS ADDRESSING ON ALL HWYS AND STREETS.*  
- `n_comments_subtree`: how many comments are in the subtree of the key point (matched to the key point or any of its descendants).  


In addition to the individual key points, in the last rows we can find the statistics of total and matched number of sentences and comments for each stance, starting with \*: in the current example, there are 1000 comments and 1813 sentences in total, 668 and 920 of them are matched to at least one key point, respectively. Out of those, 881 comments have sentences classified as con, and overall 1450 sentences are classified as con. 655 comments and 898 sentences are matched to con key points.

### 3.2 Docx report
A Microsoft Word document that shows the key point heirarchy visually and presents the sentences matched to each key point as a user-friendly report. it can be generated using:

In [17]:
kps_result_2016.generate_docx_report(output_dir = "kps_results", result_name="kps_result_2016")

2023-06-04 23:22:48,575 [INFO] docx_generator.py 208: Creating key points hierarchy
2023-06-04 23:22:48,600 [INFO] docx_generator.py 216: Creating key points matches tables
2023-06-04 23:22:48,908 [INFO] docx_generator.py 292: saving docx summary in file: kps_results/kps_result_2016_hierarchical.docx


Note that this report displays only the more salient key points (that are matched to at least 5 sentneces) so there might be some differences from the summary report. 

### 3.3 Full report

The attribute *result_df* stores the full results. Each row stores a pair of a key point, a matched sentence, their match_score and all the information regarding the sentence.

In [16]:
kps_result_2016.result_df.head(3)

,kp,sentence_text,match_score,comment_id,sentence_id,sents_in_comment,span_start,span_end,num_tokens,argument_quality,kp_quality,sent_kp_quality,pos_score,neg_score,sug_score,neut_score,selected_stance,stance_conf,kp_stance
0,Improve traffic flow.,Improve traffic flow.,1.000000,269,0,1,0,21,3,0.310709,0.995855,0.995855,0.001960,0.003750,0.988809,0.005481,sug,0.988809,con
1,Improve traffic flow.,The traffic on Cameron Road was worsened by th...,0.999983,674,0,3,0,71,13,0.603956,0.995855,0.000000,0.001241,0.995374,0.001376,0.002009,neg,0.995374,con
2,Improve traffic flow.,Fix the traffic - you keep encouraging growth ...,0.999983,210,0,2,0,108,17,0.626678,0.995855,0.000000,0.000969,0.989541,0.004868,0.004622,neg,0.989541,con


It's also possible to generate all three reports together. You need to provide the output_dir and the result name, and they will be written to file with the appropriate suffixes:

In [19]:
kps_result_2016.export_to_all_outputs(output_dir="kps_results", result_name="kps_result_2016")

2023-06-04 23:23:34,419 [INFO] utils.py 60: Writing dataframe to: kps_results/kps_result_2016.csv
2023-06-04 23:23:34,442 [INFO] utils.py 60: Writing dataframe to: kps_results/kps_result_2016_kps_summary.csv
2023-06-04 23:23:34,492 [INFO] docx_generator.py 208: Creating key points hierarchy
2023-06-04 23:23:34,515 [INFO] docx_generator.py 216: Creating key points matches tables
2023-06-04 23:23:34,822 [INFO] docx_generator.py 292: saving docx summary in file: kps_results/kps_result_2016_hierarchical.docx


### 3.4 Unmatched sentences #### TODO CANT BE USED WITH THE FULL FLOW SINCE DOMAIN IS DELETED!!! WHERE TO PUT THIS PART????
Note that the KpsResult object only stores the sentences that were matched to at least one key point. In order to get the list of sentences that were not matched to any key point, the client needs to be called with the KpsResult object as a parameter:

In [21]:
unmapped_sentences_df = keypoints_client.get_unmapped_sentences_for_kps_result(kps_result_2016)
unmapped_sentences_df.to_csv("kps_results/kps_result_merged_unmapped_sentences.csv")

2023-06-04 23:25:05,030 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-04 23:25:05,784 [INFO] keypoints_client.py 486: no sentences found for domain austin_demo_full
2023-06-04 23:25:05,786 [INFO] keypoints_client.py 489: No sentences found, returning empty dataframe.
2023-06-04 23:25:05,792 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-04 23:25:06,436 [INFO] keypoints_client.py 486: no sentences found for domain austin_demo_full
2023-06-04 23:25:06,439 [INFO] keypoints_client.py 489: No sentences found, returning empty dataframe.
2023-06-04 23:25:06,445 [INFO] keypoints_client.py 466: No sentences found for result, maybe the domain was deleted?


AttributeError: 'NoneType' object has no attribute 'to_csv'

More advanced capabiliteis of the KpaResult, such as comparing within and between summarization jobs, will be presented later. 

## 4 Run Key Point Summarization in a stagged manner.  
In order to customize the key points summerization service and fully exploit its caching and comparitive capabilities, we must run the service in a stagged manner. Let's dive into each step and understand the KPS flow.  

### 4.1 Create a domain
The *Key Point Summarization* service stores the data (and cached-results) in a *domain*. A user can create several domains, one for each dataset. Domains are only accessible to the user who created them.

Create a domin using the **keypoints_client.create_domain(domain=domain, domain_params={})** method. Several params can be passed when creating a domain in the domain_params dictionary as described in the documentation. Leaving it empty gives us a good default behaviour. 
By deafult, an exception is thrown if the domain already exists. To avoid this exception, add the parameter **ignore_exists=True** to the method. Note that in this case the domain_params are not updated and the existed domain remains the same. 

In this tutorial we will first delete the domain to make sure that we start with an empty domain.

Full documentation of the supported *domain_params* can be found [here](kps_parameters.pdf).

In [22]:
domain = 'austin_demo'
keypoints_client.delete_domain_cannot_be_undone(domain=domain)
keypoints_client.create_domain(domain=domain, domain_params={})

2023-06-04 23:32:47,097 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-04 23:32:47,660 [ERROR] keypoints_client.py 77: There is a problem with the request (422): user: db0a12 doesn't have domain: austin_demo
2023-06-04 23:32:47,661 [INFO] keypoints_client.py 413: domain: austin_demo doesn't exist.
2023-06-04 23:32:47,663 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/domains
2023-06-04 23:32:48,585 [INFO] keypoints_client.py 113: created domain: austin_demo with domain_params: {}


Few domain related points:
* The domain must be comprised of alphanumeric characters, spaces, or underscores. 
* We can always delete a domain we no longer need using: **keypoints_client.delete_domain_cannot_be_undone(domain=domain)**
* Each domain has a state: it stores all comments previously uploaded into it and a cache with all computations performed over this data.
* If we want to restart and run over the domain from scratch (no comments and no cache), we can delete the domain and then re-create it. Keep in mind that the cache is also cleared and consecutive runs will take longer.

### 4.2 Upload comments into the domain
Upload the comments into the domain using the **keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids, comments_texts=comments_texts)** method. This method receives the domain, a list of comment_ids and a list of comment_texts. When uploading comments into a domain, the *Key Point Summarization* service splits the comments into sentences and runs a minor cleansing on them. If you have domain-specific knowladge and want to split the comments into sentences or clean them yourself, you can use the *split_comments* or *clean_comments* domain_params when creating the domain to disable this functionality in KPS. 

Note that:
* Comments_ids must be unique strings comprised of alphanumeric characters, spaces or underscores.
* The number of comments_ids must match the number comments_texts
* Comments_texts must not be longer than 3000 characters
* Uploading the same comment several times (same domain + comment_id + comment_text) is not a problem and the comment is only processed once.
* Uploading the same comment_id with a different comment_text will raise an exception. 

After being uploaded to the domain, the comments are processed. The method runs in a synchronous manner and returns only after all the comments are processed. In the meanwhile, the status of the processing is printed on screen.
For information about running KPS in an async manner, see section 7.

In [23]:
comments_texts_2016 = list(comments_2016_sample_df['text'])
comments_ids_2016 = list(comments_2016_sample_df['id'].astype(str))
keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids_2016, comments_texts=comments_texts_2016)

2023-06-04 23:37:26,308 [INFO] keypoints_client.py 158: uploading 1000 comments in batches
2023-06-04 23:37:26,314 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 23:37:27,437 [INFO] keypoints_client.py 174: uploaded 1000 comments, out of 1000
2023-06-04 23:37:27,438 [INFO] keypoints_client.py 137: waiting for the comments to be processed
2023-06-04 23:37:27,440 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 23:37:28,045 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 0, 'processed_sentences': 0, 'pending_comments': 1000}
2023-06-04 23:37:38,052 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 23:37:38,666 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_com

To examine the processed data, we can download the processed sentences and save them into a csv:

In [24]:
sentences_df = keypoints_client.get_sentences_for_domain(domain=domain)
sentences_df.to_csv(f"kps_results/{domain}_sentences.csv")

2023-06-04 23:38:47,701 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-04 23:38:49,603 [INFO] keypoints_client.py 486: returning 1813 sentences for domain austin_demo


### 4.3 Run a KPS job
Run a *Key Point Summarization* job using the **keypoints_client.run_kps_job(domain=domain)** method. 
This method receives:
* The domain.
* Optional *comment_ids*: by default, the summarization is performed over all comments in the domain. If we need to run over a subset of the comments (split the data by different GEOs/users types/timeframes etc') we can pass a list of thier comments_ids.
* Optional *run_params*: a dictionary with various parameters for customizing the job (see section 4.4).
* Optional *stance*: unlike in the run_full_kps_flow method presented in section 2, here no stance analysis is performed by default. see section 4.5 for stance customization.
* Optional *description*: a description of the job to appear in the user report (see section 5.1).  

The system extracts the key points from the input comments, and matches each sentence in the comments with all its matching key points.
The job runs in a synchronous manner, prints the progress to the screen and returns the results eventually.   
For information about running KPS in an async manner, see section 7. 

In [26]:
kps_result_2016_no_stance = keypoints_client.run_kps_job(domain = domain)

2023-06-04 23:50:09,734 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 23:50:10,354 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 1000, 'processed_sentences': 1813, 'pending_comments': 0}
2023-06-04 23:50:10,355 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:50:11,053 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: no-stance, run_params: None, job_id: 647cf903f0128c554c89b81d
2023-06-04 23:50:11,055 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:50:11,623 [INFO] keypoints_client.py 631: job_id 647cf903f0128c554c89b81d is pending
2023-06-04 23:50:41,629 [INFO] keypoints_client.py 66: client calls service (get): https://keypoi

Stage 1/2: |--------------------------------------------------| 0.0% Complete



2023-06-04 23:51:12,283 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:51:12,915 [INFO] keypoints_client.py 635: job_id 647cf903f0128c554c89b81d is running, progress: {'total_stages': 2, 'stage_1': {'inferred_batches': 6, 'total_batches': 17, 'batch_size': 2000}}


Stage 1/2: |█████████████████---------------------------------| 35.3% Complete



2023-06-04 23:51:42,921 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:51:43,613 [INFO] keypoints_client.py 635: job_id 647cf903f0128c554c89b81d is running, progress: {'total_stages': 2, 'stage_1': {'inferred_batches': 16, 'total_batches': 17, 'batch_size': 2000}}


Stage 1/2: |███████████████████████████████████████████████---| 94.1% Complete



2023-06-04 23:52:13,620 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:52:14,246 [INFO] keypoints_client.py 635: job_id 647cf903f0128c554c89b81d is running, progress: {'total_stages': 2, 'stage_1': {'inferred_batches': 17, 'total_batches': 17, 'batch_size': 2000}, 'stage_2': {'inferred_batches': 0, 'total_batches': 2, 'batch_size': 2000}}


Stage 2/2: |--------------------------------------------------| 0.0% Complete



2023-06-04 23:52:44,252 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:52:46,069 [INFO] keypoints_client.py 638: job_id 647cf903f0128c554c89b81d is done, returning result


Let's print the results: 

In [27]:
kps_result_2016_no_stance.print_result(n_sentences_per_kp = 3, title = "Austin sample 2016 full flow no stance", n_top_kps = 20)

Austin sample 2016 full flow no stance results, stance: no-stance 
Coverage (all comments): 74.50
Displaying 20 key points out of 63:
203 - Improve Austin's traffic!
	- Fix the traffic - you keep encouraging growth of the city without the infrastructure to
	  support that growth.
	- Get the bikes off the streets and improve traffic flow in and out of the city.
	- Austin should use dedicated bus lanes and bicycle lanes like Atlanta, Georgia and solve
	  traffic issues.
194 - We really need to improve our public transportation.
	- Need to rework zoning to improve traffic and add more public transit, especially to
	  suburbs.
	- The city needs a more comprehensive and better connected transportation network
	  including bus, rail, bike, pedestrian access.
	- Austin needs to get serious about alternatives to driving including real mass transit,
	  bike, and pedestrian facilities.
187 - fix the roads so automobiles can travel easier.
	- Please fix the traffic flow in and around this city.
	

### 4.4 Modify the run_params to customize the summarization
Each domain has a cache that stores all intermediate results that are calculated during the summarization. Therefore modifing the run_params and running another summarization is usually faster since all intersecting intermediate results are retreived from cache. 

Full documentation of the supported *run_params* can be found [here](kps_parameters.pdf).
Some of the notable options:
* By default, key points are extracted automatically. When we want to provide key points and match all sentences to these key points we can do so by passing them to the keypoints parameter: **run_param['key_points'] = [...]**. This enables a mode of work named human-in-the-loop where we first automatically extract key points, then we manually edit them (refine non-perfect key points, remove duplicated and add missing ones) and then run again, this time providing the edited keypoints as a given set of key points.
* It is also possible to provide key points and let KPS add additional missing key points. To do so pass the key points to the key_point_candidates parameter: **run_param['key_point_candidates'] = [...]** (see section 6 for an elaborated example).
* Change the lengths of the required key points and the sentences participating in the summarization.
* The **mapping_policy** is used when mapping all sentences to the final key points: the default value is **NORMAL**. Changing to **STRICT** will cause only the sentence and key point pairs with very high matching confidence to be considered matched, increasing precision but potentially decreasing coverage. Changing it to **LOOSE** will do the opposite and match pairs with lower confidene. 

Let's run with 'LOOSE' mapping policy:

In [29]:
run_params = {'mapping_policy':'LOOSE'}
kps_result_loose = keypoints_client.run_kps_job(domain=domain, run_params=run_params)
kps_result_loose.print_result(n_sentences_per_kp=2, title='Austin sample 2016 LOOSE', n_top_kps = 20)

2023-06-04 23:55:56,767 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-04 23:55:57,359 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 1000, 'processed_sentences': 1813, 'pending_comments': 0}
2023-06-04 23:55:57,362 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:55:58,032 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: no-stance, run_params: {'mapping_policy': 'LOOSE'}, job_id: 647cfa5ef0128c554c89b81f
2023-06-04 23:55:58,035 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-04 23:55:58,610 [INFO] keypoints_client.py 631: job_id 647cfa5ef0128c554c89b81f is pending
2023-06-04 23:56:28,619 [INFO] keypoints_client.py 66: client calls servic

Austin sample 2016 LOOSE results, stance: no-stance 
Coverage (all comments): 82.30
Displaying 20 key points out of 66:
253 - We really need to improve our public transportation.
	- Need to rework zoning to improve traffic and add more public transit, especially to
	  suburbs.
	- The city needs a more comprehensive and better connected transportation network
	  including bus, rail, bike, pedestrian access.
242 - fix the roads so automobiles can travel easier.
	- Please fix the traffic flow in and around this city.
	- Need to fix traffic congestion in the city highways urgently.
218 - Improve Austin's traffic!
	- Fix the traffic - you keep encouraging growth of the city without the infrastructure to
	  support that growth.
	- Get the bikes off the streets and improve traffic flow in and out of the city.
198 - Improve affordable housing/living.
	- Affordable housing is also a problem- the rents are getting too high and are hard for
	  families to afford.
	- The city needs to play a stron

By changing the mapping policy to **LOOSE** the comments' coverage was increased from 74.5% to 82%.

### 4.5 Add stance analysis to the summarization

In many usecases (surveys, customer feedback, etc.) the comments have positive and/or negative stance, and it is usful to create a KPS on each stance seperatly. Most stance detection models don't perform too well on survey data (also costumer feedbacks etc.) since the comments tend to have many "suggestions" in them, and the suggestions tend to appear as positive to the model while the user suggests to improve something that needs improvement. For that end we trained a stance-model that handles suggestions well and labels each sentence as 'Positive', 'Negative', 'Neutral' and 'Suggestion'. We treat Suggestions as negatives and run two separate summarizations, first over 'Positive' sentences (pro) and second over 'Negative' and 'Suggestions' sentences (con).

This has the following advantages:

* Creates a separate positive/negative summary that shows clearly what works well and what needs to be improved.
* Filters-out neutral sentences that usually don't contain valuable information.
* Helps the matching model avoid stance mistakes (matching a positive sentence to a negative key point and vice-versa).

In order to run for each stance seperately, use the **stance** parameter in *run_kps_job*. the options are either "pro", "con", or "no-stance" (default).
First, let's run on *pro* sentences:

In [14]:
kps_results_2016_pro = keypoints_client.run_kps_job(domain=domain, stance="pro")
kps_results_2016_pro.print_result(n_sentences_per_kp=2, title='Random sample pro')

2023-06-01 23:11:49,885 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-01 23:11:50,515 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}
2023-06-01 23:11:50,519 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-01 23:11:51,161 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: pro, run_params: {'stance': 'PRO'}, job_id: 6478fb87f0128c554c89afcc
2023-06-01 23:11:51,162 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-01 23:11:51,712 [INFO] keypoints_client.py 631: job_id 6478fb87f0128c554c89afcc is pending
2023-06-01 23:12:21,717 [INFO] keypoints_client.py 66: client calls service (get): https://k

Random sample pro coverage (all sentences): 0.28
Random sample pro coverage (of pro sentences): 6.06
Random sample pro key points:
2 - City services (water, streets, electric) are outstanding!! - pro
	- RESIDENTIAL SERVICES ARE EXCELLENT!


As you can see, only 0.28% of the sentences are positive sentences that were matched to a key point, and only 6% of the positive sentences are matched. As in many surveys, most comments are negative/suggestions. The positive comments are too few to extract meaningful key points and the results are therefore very limited. In order to generate better positive results, we can try to run a the full dataset.

Let's now run for *con*:

In [15]:
kps_results_2016_con = keypoints_client.run_kps_job(domain=domain, stance="con")
kps_results_2016_con.print_result(n_sentences_per_kp=2, title='Random sample con')

2023-06-01 23:13:30,348 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-01 23:13:30,942 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}
2023-06-01 23:13:30,945 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-01 23:13:31,596 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: con, run_params: {'stance': 'CON'}, job_id: 6478fbebf0128c554c89afd0
2023-06-01 23:13:31,599 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-01 23:13:32,178 [INFO] keypoints_client.py 631: job_id 6478fbebf0128c554c89afd0 is pending
2023-06-01 23:14:02,185 [INFO] keypoints_client.py 66: client calls service (get): https://k

Random sample con coverage (all sentences): 50.34
Random sample con coverage (of con sentences): 62.78
Random sample con key points:
102 - TRAFFIC NEEDS ADDRESSING ON ALL HWYS AND STREETS - con
	- Need to fix traffic congestion in the city highways urgently.
	- Fix the traffic - you keep encouraging growth of the city without the infrastructure to
	  support that growth.
66 - Improve affordable housing/living. - con
	- Cost of living here is to high & tax for my home is to high.
	- Reduce property taxes and housing costs so that retiring and still living here is a real
	  possibility.
54 - Public transportation needs to improve. - con
	- Need to rework zoning to improve traffic and add more public transit, especially to
	  suburbs.
	- There needs to be better and more public transportation in the city of Austin.
51 - BIKE LANES HURT TRAFFIC. - con
	- Remove peddlers at traffic lights and fix traffic problems.
	- FIX THE TRAFFIC.
33 - PROPERTY TAXES ARE TOO HIGH. - con
	- Consider the f

We can create a new unified kps_result by combining the pro and the con results. This new result will keep the information about both pro and con key points and sentences. It's important to only merge pro and con results that were obtained over the same domain and using the same set of comments, otherwise an error will be raised. 
Let's combine the pro and con results to a single results, and print it to the screen:

In [19]:
kps_result_2016_merged = KpsResult.get_merged_pro_con_results(pro_result=kps_results_2016_pro, con_result=kps_results_2016_con)
kps_result_2016_merged.print_result(n_sentences_per_kp=2, title='Random sample con')

Random sample con coverage (all sentences): 50.62
Random sample con coverage (of pro and con sentences): 59.74
Random sample con key points:
102 - TRAFFIC NEEDS ADDRESSING ON ALL HWYS AND STREETS - con
	- Need to fix traffic congestion in the city highways urgently.
	- Fix the traffic - you keep encouraging growth of the city without the infrastructure to
	  support that growth.
66 - Improve affordable housing/living. - con
	- Cost of living here is to high & tax for my home is to high.
	- Reduce property taxes and housing costs so that retiring and still living here is a real
	  possibility.
54 - Public transportation needs to improve. - con
	- Need to rework zoning to improve traffic and add more public transit, especially to
	  suburbs.
	- There needs to be better and more public transportation in the city of Austin.
51 - BIKE LANES HURT TRAFFIC. - con
	- Remove peddlers at traffic lights and fix traffic problems.
	- FIX THE TRAFFIC.
33 - PROPERTY TAXES ARE TOO HIGH. - con
	- Consid


Alternatively, we can avoid running a seperate summarization for each stance and use the *run_kps_job_both_stances*.
This method starts two seperate jobs simultenously, one for *pro* and one for *con*. It later retreives the results, unify them and returns the merge results object.
This method recieves: 
* The domain
* comments_ids - optional, as in *run_kps_job*
* desription - optional, as in *run_kps_job*, stance is appended to the description of each job
* run_params_pro - optional run_params to be sent to the *pro* job
* run_params_con - optional run_params to be sent to the *con* job

In [16]:
kps_result_2016_merged_2 = keypoints_client.run_kps_job_both_stances(domain)

2023-06-01 23:26:08,018 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-01 23:26:08,659 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}
2023-06-01 23:26:08,663 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-01 23:26:09,299 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: pro, run_params: {'stance': 'PRO'}, job_id: 6478fee1f0128c554c89afe7
2023-06-01 23:26:09,302 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-01 23:26:09,906 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}
2023-06-01 23:26:09,907

## 5. Jobs management

### 5.1 User report
The user report stores all the information about existing domains and all past and present kps jobs. 
To fetch it and print to screen:

In [ ]:
report = keypoints_client.get_full_report()
keypoints_client.print_report(report)

### 5.2 job_id
Each job has a unique job_id which is useful for the jobs managements, and can be obtained in several ways:
* It's printed to the screen when the job starts and in every progress update.
* From the user report.
* When running asyncronously (see section 7)
* From the KpsResult object, using the **get_stance_to_job_id** method:

In [33]:
print(kps_result_2016.get_stance_to_job_id())

{'pro': '647ce79cf0128c554c89b7ef', 'con': '647ce79ef0128c554c89b7f0'}


In [34]:
print(kps_result_2016_no_stance.get_stance_to_job_id())

{'no-stance': '647cf903f0128c554c89b81d'}


### 5.3 Canceling a job
Simply exiting the program after the job is sent does not cancel the job, since it keeps running on the server, consuming resources. In order to cancel a job, use: 
* **keypoints_client.cancel_kp_extraction_job(\<job_id\>)**

It is also possibe to stop all jobs in a domain, or even all jobs in all domains (might be simpler since there is no need of the job_id):

* **keypoints_client.cancel_all_extraction_jobs_for_domain(\<domain\>)**
* **keypoints_client.cancel_all_extraction_jobs_all_domains()**


### 5.4 fetching the results of a previous job
If the program terminated unexpectedly after the job was sent, you can still fetch the results using:
* **kps_result = keypoints_client.get_results_from_job_id(\<job_id\>)**

## 6. Run *Key Point Summarization* incrementally
### 6.1 Run *Key Point Summarization* incrementally on new data (data from 2016 + 2017)
A year passed, and we collect additional data (data from 2017). We can now upload the 2017 data to the same domain (austin_demo) and have both 2016 and 2017 data in one domain. 

In [27]:
comments_2017_df = comments_df[comments_df['year'] == 2017]

sample_size = 400
comments_2017_sample_df = comments_2017_df.sample(n = sample_size, random_state = 1)

domain = 'austin_demo'
comments_texts_2017 = list(comments_2017_sample_df['text'])
comments_ids_2017 = list(comments_2017_sample_df['id'].astype(str))
keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids_2017, comments_texts=comments_texts_2017)

2023-06-02 00:18:39,925 [INFO] keypoints_client.py 158: uploading 400 comments in batches
2023-06-02 00:18:39,926 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:18:40,878 [INFO] keypoints_client.py 174: uploaded 400 comments, out of 400
2023-06-02 00:18:40,881 [INFO] keypoints_client.py 137: waiting for the comments to be processed
2023-06-02 00:18:40,883 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:18:41,522 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 400}
2023-06-02 00:18:51,528 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:18:52,155 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_com

We can now run a new summarization over all the data in the domain, as we did before, and automatically extract new key points. We can assume that some will be identical to the key points extracted on the 2016 data, some will be similar and some key points will be new.

A better option is to run a new summarization but provide the keypoints from the 2016 summarization and let *Key Point Summarization* add new key points from the 2017 data if there are such. One benefit of this approach is that the new result will mostly use 2016 key point and we will be able to compare between them and view the trends. Another major benefit for this approach is run-time. 2016 data was already analyzed with these key points and since we have a cache in place, much of the computation can be avoided. The 2016 key points can be provided via the: **run_param['key_point_candidates'] = [...]** parameter, passing a list of strings, or we can use: **run_param['key_point_candidates_by_job_ids'] = [\<job_id1\>,...]** and provide a list of previous job_ids. KPS will take the key points from the jobs' result automatically.

For simplicity, we'll run over the result without the stance analysis. We can also use the incremental approach when running on both stances: we will need to provide the job_id of the positive summarization of 2016 in the run_params_pro and the job_id of negative summarization of 2016 in the run_params_con when running on all sentences from 2016+2017.

First, let's extract the job id from the result:

In [28]:
stance_to_job_id = kps_result_2016.get_stance_to_job_id()
print(stance_to_job_id)
job_id_2016 = stance_to_job_id["no-stance"]

{'no-stance': '6478f992f0128c554c89afbd'}


In [29]:
run_params = {'key_point_candidates_by_job_ids': [job_id_2016]}
kps_result_2016_2017 = keypoints_client.run_kps_job(domain, run_params=run_params)

2023-06-02 00:19:56,589 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:19:57,173 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 800, 'processed_sentences': 1510, 'pending_comments': 0}
2023-06-02 00:19:57,176 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:19:57,882 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: no-stance, run_params: {'key_point_candidates_by_job_ids': ['6478f992f0128c554c89afbd']}, job_id: 64790b7df0128c554c89b04a
2023-06-02 00:19:57,884 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:19:58,467 [INFO] keypoints_client.py 631: job_id 64790b7df0128c554c89b04a is pending
2023-06-02 00:20:28,475 [INFO] keypoi

Stage 1/3: |--------------------------------------------------| 0.0% Complete



2023-06-02 00:21:29,735 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:21:30,371 [INFO] keypoints_client.py 635: job_id 64790b7df0128c554c89b04a is running, progress: {'total_stages': 3, 'stage_0': {'inferred_batches': 5, 'total_batches': 5, 'batch_size': 2000}, 'stage_1': {'inferred_batches': 3, 'total_batches': 8, 'batch_size': 2000}}


Stage 1/3: |██████████████████--------------------------------| 37.5% Complete



2023-06-02 00:22:00,378 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:22:00,978 [INFO] keypoints_client.py 635: job_id 64790b7df0128c554c89b04a is running, progress: {'total_stages': 3, 'stage_0': {'inferred_batches': 5, 'total_batches': 5, 'batch_size': 2000}, 'stage_1': {'inferred_batches': 8, 'total_batches': 8, 'batch_size': 2000}}


Stage 1/3: |██████████████████████████████████████████████████| 100.0% Complete




2023-06-02 00:22:30,988 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:22:32,896 [INFO] keypoints_client.py 638: job_id 64790b7df0128c554c89b04a is done, returning result


Now, we can compare the results obtained by the 2016+2017 data with the results obtained in 2016, using the method **kps_result_2016_2017.compare_with_comment_subsets()**.
This method recieves a dictionary with mappings from subset names to sets of comment ids, and performs the comparison between the full result and the subset results. For the full result and for each of the subsets, you can see the number and percentage of the comments that match each key point. 

In [32]:
subsets_dict = {"2016":comments_ids_2016, "2017":comments_ids_2017}
comparison_df = kps_result_2016_2017.compare_with_comment_subsets(subsets_dict)
comparison_df

,key point,full_n_comments,full_precent,2016_n_comments,2016_precent,2017_n_comments,2017_precent
0,TRAFFIC NEEDS ADDRESSING ON ALL HWYS AND STREETS,177,22.12%,98,24.50%,79,19.75%
1,Improve affordable housing/living.,145,18.12%,60,15.00%,85,21.25%
2,Public transportation needs to improve.,102,12.75%,56,14.00%,46,11.50%
3,BIKE LANES HURT TRAFFIC.,94,11.75%,58,14.50%,36,9.00%
4,PROPERTY TAXES ARE TOO HIGH.,77,9.62%,33,8.25%,44,11.00%
5,Integrated transportation is critical.,57,7.12%,37,9.25%,20,5.00%
6,Buying a home in S Austin is prohibitive.,54,6.75%,19,4.75%,35,8.75%
7,"Utility ""fees"" create great hardship on the poor.",41,5.12%,16,4.00%,25,6.25%
8,Don't let Austin become Houston with overdevel...,37,4.62%,17,4.25%,20,5.00%
9,Water costs too much.,37,4.62%,17,4.25%,20,5.00%


Using this method, we can compare results over subsets of the data in the same domain. The subsets can be data from different GEOs, different organizations, different times, different users (e.g. promoters/detractors) etc.).

### 5.2 Run *Key Point Summarization* incrementaly on new data (2017 independantly)
If you don't care about the 2016+2017 combination and only want to compare the 2016 and the 2017 data, You can use the **comments_ids** parameter in the **run_kps_job** method, to run on a subset of the comments in the domain. Let's do that and run summarization over 2017 comments independantly. We will provide the key points from 2016 since we want to able to compare between them:

In [33]:
run_params = {'key_point_candidates_by_job_ids': [job_id_2016]}
kps_result_2017 = keypoints_client.run_kps_job(domain, run_params=run_params, comments_ids = comments_ids_2017)

2023-06-02 00:26:10,536 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:26:11,123 [INFO] keypoints_client.py 187: domain: austin_demo, comments status: {'processed_comments': 800, 'processed_sentences': 1510, 'pending_comments': 0}
2023-06-02 00:26:11,126 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:26:11,971 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo, stance: no-stance, run_params: {'key_point_candidates_by_job_ids': ['6478f992f0128c554c89afbd']}, job_id: 64790cf4f0128c554c89b04b
2023-06-02 00:26:11,975 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:26:12,561 [INFO] keypoints_client.py 631: job_id 64790cf4f0128c554c89b04b is pending
2023-06-02 00:26:42,568 [INFO] keypoi

Stage 1/3: |█████████████████████████-------------------------| 50.0% Complete



2023-06-02 00:27:44,162 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:27:44,772 [INFO] keypoints_client.py 635: job_id 64790cf4f0128c554c89b04b is running, progress: {'total_stages': 3, 'stage_0': {'inferred_batches': 4, 'total_batches': 4, 'batch_size': 2000}, 'stage_1': {'inferred_batches': 1, 'total_batches': 2, 'batch_size': 2000}}


Stage 1/3: |█████████████████████████-------------------------| 50.0% Complete



2023-06-02 00:28:14,775 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:28:16,506 [INFO] keypoints_client.py 638: job_id 64790cf4f0128c554c89b04b is done, returning result


Now, we can compare the kps_result_2016 with the kps_result_2017 using the *compare_with_other_results* method. This method receives the title for the current results and a dictionary mapping from results name to KpsResult obects, and returns the comparison table. If the comparison is with a single other result, the change percent is also displayed. 

In [34]:
kps_result_2016.compare_with_other_results(this_title="2016", other_results_dict = {"2017":kps_result_2017})

,key point,2016_n_comments,2016_precent,2017_n_comments,2017_precent,change_percent
0,TRAFFIC NEEDS ADDRESSING ON ALL HWYS AND STREETS,99,24.75%,80,20.00%,-4.75%
1,Improve affordable housing/living.,60,15.00%,86,21.50%,6.50%
2,BIKE LANES HURT TRAFFIC.,58,14.50%,36,9.00%,-5.50%
3,Public transportation needs to improve.,56,14.00%,47,11.75%,-2.25%
4,Integrated transportation is critical.,37,9.25%,21,5.25%,-4.00%
5,PROPERTY TAXES ARE TOO HIGH.,33,8.25%,44,11.00%,2.75%
6,Buying a home in S Austin is prohibitive.,21,5.25%,35,8.75%,3.50%
7,Don't let Austin become Houston with overdevel...,18,4.50%,20,5.00%,0.50%
8,Water costs too much.,17,4.25%,20,5.00%,0.75%
9,Keep Austin clean & in compliance.,17,4.25%,12,3.00%,-1.25%


This allows us to compare the results of different kps jobs that were using the same set of key points. You can see, for example, that in 2017 people complained more than in 2016 about affordable housing, and less about the traffic. 

## 7. Running KPS in async mode

It is also possible to upload comments and run kps jobs in an asynchronous manner. This can be useful when you want to start several jobs simultaneously, and then later collect the results.

Let's create a new domain for sake of the demonstaration. Note that this is not required, as async and sync calls can be used on the same doamin.

In [35]:
domain = 'austin_demo_async'
keypoints_client.delete_domain_cannot_be_undone(domain=domain)
keypoints_client.create_domain(domain=domain, domain_params={})

2023-06-02 00:31:26,934 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-02 00:31:27,457 [ERROR] keypoints_client.py 77: There is a problem with the request (422): user: db0a12 doesn't have domain: austin_demo_async
2023-06-02 00:31:27,461 [INFO] keypoints_client.py 413: domain: austin_demo_async doesn't exist.
2023-06-02 00:31:27,464 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/domains
2023-06-02 00:31:28,425 [INFO] keypoints_client.py 113: created domain: austin_demo_async with domain_params: {}


### 7.1 Uploading comments asynchronously

In order to start loading comments, use the *upload_comments_async* method: 

In [36]:
comments_texts = list(comments_2016_sample_df['text'])
comments_ids = list(comments_2016_sample_df['id'].astype(str))
keypoints_client.upload_comments_async(domain=domain, comments_ids=comments_ids, comments_texts=comments_texts)

2023-06-02 00:33:42,989 [INFO] keypoints_client.py 158: uploading 400 comments in batches
2023-06-02 00:33:42,996 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:33:43,956 [INFO] keypoints_client.py 174: uploaded 400 comments, out of 400


The method uploads the comments and returns immediately. We must wait until all comments finish processing before starting a kps job. This can be checked via the *are_all_comments_processed* method, which prints the upload status and returns True when the domain is ready for jobs:

In [37]:
res = keypoints_client.are_all_comments_processed(domain)
print(res)

2023-06-02 00:33:47,762 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:33:48,374 [INFO] keypoints_client.py 187: domain: austin_demo_async, comments status: {'processed_comments': 0, 'processed_sentences': 0, 'pending_comments': 400}


False


You can also use the keypoints_client.wait_till_all_comments_are_processed(domain=domain) method, that returns only after the comments are processed:

In [38]:
keypoints_client.wait_till_all_comments_are_processed(domain=domain) 

2023-06-02 00:33:53,472 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:33:54,105 [INFO] keypoints_client.py 187: domain: austin_demo_async, comments status: {'processed_comments': 0, 'processed_sentences': 0, 'pending_comments': 400}
2023-06-02 00:34:04,114 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:34:04,755 [INFO] keypoints_client.py 187: domain: austin_demo_async, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}


### 7.2 Running KPS jobs asynchronously

In order to start a job in an async manner, use the *run_kps_job_async* method. This method recieves the same arguments as *run_kps_job*, but returns right after a the job is sent to the server, returning a future object:

In [39]:
future = keypoints_client.run_kps_job_async(domain=domain)

2023-06-02 00:34:09,908 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2023-06-02 00:34:10,562 [INFO] keypoints_client.py 187: domain: austin_demo_async, comments status: {'processed_comments': 400, 'processed_sentences': 727, 'pending_comments': 0}
2023-06-02 00:34:10,564 [INFO] keypoints_client.py 66: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:34:11,222 [INFO] keypoints_client.py 345: started a kp summarization job - domain: austin_demo_async, stance: no-stance, run_params: None, job_id: 64790ed3f0128c554c89b04e


Use the returned future and wait till results are available using the kps_result = future.get_result() method. The method waits for the job to finish and eventually returns the result.

In [40]:
kps_result_async = future.get_result(high_verbosity=True)

2023-06-02 00:34:23,587 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:34:24,207 [INFO] keypoints_client.py 635: job_id 64790ed3f0128c554c89b04e is running, progress: not updated yet
2023-06-02 00:34:54,216 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:34:54,820 [INFO] keypoints_client.py 635: job_id 64790ed3f0128c554c89b04e is running, progress: {'total_stages': 2, 'stage_1': {'inferred_batches': 7, 'total_batches': 7, 'batch_size': 2000}}


Stage 1/2: |██████████████████████████████████████████████████| 100.0% Complete




2023-06-02 00:35:24,824 [INFO] keypoints_client.py 66: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/kp_extraction
2023-06-02 00:35:26,604 [INFO] keypoints_client.py 638: job_id 64790ed3f0128c554c89b04e is done, returning result


The future object can also be used to obtain the job_id, via the **future.get_job_id()** method. 

## 8. Cleanup
If you finished the tutorial and no longer need the domains and the results, cleaning up is always advised:

In [41]:
keypoints_client.delete_domain_cannot_be_undone(domain='austin_demo')
keypoints_client.delete_domain_cannot_be_undone(domain='austin_demo_full')
keypoints_client.delete_domain_cannot_be_undone(domain='austin_demo_async')

2023-06-02 00:36:12,222 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-02 00:36:14,027 [INFO] keypoints_client.py 408: domain: austin_demo was deleted
2023-06-02 00:36:14,029 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-02 00:36:14,586 [ERROR] keypoints_client.py 77: There is a problem with the request (422): user: db0a12 doesn't have domain: austin_demo_full
2023-06-02 00:36:14,591 [INFO] keypoints_client.py 413: domain: austin_demo_full doesn't exist.
2023-06-02 00:36:14,592 [INFO] keypoints_client.py 66: client calls service (delete): https://keypoint-matching-backend.debater.res.ibm.com/data
2023-06-02 00:36:15,691 [INFO] keypoints_client.py 408: domain: austin_demo_async was deleted


{'status': 'success'}

## 9. Conclusion
In this tutorial, we showed how to use the *Key Point Summarization* service, and how it provides detailed insights over survey data right out of the box - significantly reducing the effort required by a data scientist to analyze the data. We also demonstrated key *key point Summarization* features such as how to modify the summarization parameters and increase coverage, how to use the stance-model and create per-stance results, and how to incrementally add new data.

Feel free to contact us for questions or assistance: *yoavka@il.ibm.com*